# High-level Keras R (TF) RNN Example

In [1]:
# SETUP
#
# Install keras R
# install.packages('keras')
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")

In [2]:
library(keras)
use_python('/anaconda/envs/py35')

# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params('lstm')

Loading required package: rjson


In [3]:
# py_config()
print(paste0("OS: ", Sys.info()["sysname"]))
print(R.version$version.string)
print(paste0("Keras: ", packageVersion("keras")))
print(paste0("Tensorflow: ", packageVersion("tensorflow")))
print(paste0("Keras using ", backend()$backend()))
print(paste0("Keras channel ordering is ", backend()$image_data_format())) 

[1] "OS: Linux"
[1] "R version 3.4.1 (2017-06-30)"
[1] "Keras: 2.0.6"
[1] "Tensorflow: 1.3.1"
[1] "Keras using tensorflow"
[1] "Keras channel ordering is channels_last"


In [4]:
create_symbol <- function(CUDNN = TRUE, maxf = params$MAXFEATURES, edim = params$EMBEDSIZE, nhid = params$NUMHIDDEN, maxl = params$MAXLEN){
    
    model <- keras_model_sequential() %>%
    
    layer_embedding(maxf,edim, input_length = maxl)
    
    if (CUDNN){
        # Not available with the current version of keras R api on dsvm
        # model %>% layer_cudnn_gru(units = nhid, return_sequences = FALSE, return_state = FALSE)
    } else{
        model %>% layer_gru(units = nhid, return_sequences = FALSE, return_state = FALSE)    
    }
    
    model %>% layer_dense(2, activation = "softmax")
    
    return(model)
}

In [5]:
init_model <- function(m, lr=params$LR, b1=params$BETA_1, b2=params$BETA_2, eps=params$EPS){
    m %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_adam(lr = lr, beta_1 = b1, beta_2 = b2, epsilon = eps),
      metrics = "accuracy"
    )
    return(m)
}

In [6]:
# Data Preparation 
imdb <- dataset_imdb(num_words = params$MAXFEATURES)

# Pad the sequences to the same length
  # This will convert our dataset into a matrix: each line is a review
  # and each column a word on the sequence
# We pad the sequences with 0s to the left
x_train <- imdb$train$x %>% pad_sequences(maxlen = params$MAXLEN)
x_test <- imdb$test$x %>% pad_sequences(maxlen = params$MAXLEN)

y_train <- to_categorical(imdb$train$y, num_classes = 2)
y_test <- to_categorical(imdb$test$y, num_classes = 2)

In [7]:
cat('x_train shape:', dim(x_train), '\n')
cat('x_test shape:', dim(x_test), '\n')
cat('y_train shape:', dim(y_train), '\n')
cat('y_test shape:', dim(y_test), '\n')

x_train shape: 25000 150 
x_test shape: 25000 150 
y_train shape: 25000 2 
y_test shape: 25000 2 


In [8]:
# Load symbol
sym = create_symbol(CUDNN = FALSE)

In [9]:
# Initialise model
model = init_model(sym)

In [10]:
# summary(model)

In [11]:
# Main training loop
model %>% fit(x_train,
          y_train,
          batch_size=params$BATCHSIZE,
          epochs=params$EPOCHS,
          verbose=1)

In [12]:
# Main evaluation loop
y_guess <- model %>% predict_classes(x_test, batch_size = params$BATCHSIZE)
y_truth <- apply(y_test, 1, function(x) which.max(x)-1)

In [13]:
print(paste0("Accuracy: ", sum(y_guess == y_truth)/length(y_guess)))

[1] "Accuracy: 0.84556"
